In [ ]:
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import argparse
import skimage, skimage.io
import torch
import torchvision

import torchxrayvision as xrv

: 

In [ ]:
img = skimage.io.imread("../Datasets/images/person14_virus_44.jpeg")
img = xrv.datasets.normalize(img, 255)  

# Check that images are 2D arrays
if len(img.shape) > 2:
    img = img[:, :, 0]
if len(img.shape) < 2:
    print("error, dimension lower than 2 for image")

# Add color channel
img = img[None, :, :]

: 

In [ ]:
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                            xrv.datasets.XRayResizer(224)])

img = transform(img)
img = torch.from_numpy(img).unsqueeze(0)

: 

In [ ]:
model = xrv.baseline_models.jfhealthcare.DenseNet()


model.pathologies

: 

In [ ]:
plt.imshow(img[0][0], cmap='gray');

: 

In [ ]:
target = model.pathologies.index('Lung Opacity')

: 

In [ ]:
img = img.requires_grad_()

outputs, logit_map = model(img)
print(logit_map)
grads = torch.autograd.grad(outputs[:,target], img)[0][0][0]
blurred = skimage.filters.gaussian(grads.detach().cpu().numpy()**2, sigma=(5, 5), truncate=3.5)

my_dpi = 200
fig = plt.figure(frameon=False, figsize=(1000/my_dpi, 1000/my_dpi), dpi=my_dpi)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(img[0][0].detach().cpu().numpy(), cmap="gray", aspect='auto')
ax.imshow(blurred, alpha=0.5)
fig.savefig("../../Cpp/a.png")

: 